1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-12-01 09:34:15--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-12-01 09:34:15--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc493581686bdc35bbb96fa9aa1b.dl.dropboxusercontent.com/cd/0/inline/Bxx7rOe__g29UctBXBl_EG_wHvUbRFgQAv-Xxhalym9IwvFMIGvqxeI_HJ0Wv61afo4-ec8IjDnE0rIFbnd8IfwrhFDG8QceyTFXTnGxqneOGgali21WF3yazEVB-_rLHe0PHDS3wqPA3UEsxnrzpBBo8Gwxx_QLYQ3GciP9N61o1Q/file# [following]
--2022-12-01 09:34:15--  https://uc493581686bdc35bbb96fa9aa1b.dl.dropboxusercontent.com/cd/0/inline/Bxx7rOe__g29UctBXBl_EG_wHvUbRFgQAv-Xxhalym9IwvFMIGvqxeI_HJ0Wv61afo4-ec8

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
import numpy as np
import pandas as pd
file_name = "./data/data_000637.txt"
N=11000
data = pd.read_csv(file_name, nrows = N)
data.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [5]:
x=data['BX_COUNTER'].max()
print('The number of BX in a ORBIT:',x)

The number of BX in a ORBIT: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [6]:
data['Time_comb']=data['BX_COUNTER']*(1+x)*25 + data['TDC_MEAS']*(25/30) 
data['Time_comb'] = pd.to_datetime(data['Time_comb'])
data.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time_comb
0,1,0,123,3869200167,2374,26,1970-01-01 00:00:00.211523421
1,1,0,124,3869200167,2374,27,1970-01-01 00:00:00.211523422
2,1,0,63,3869200167,2553,28,1970-01-01 00:00:00.227472323
3,1,0,64,3869200167,2558,19,1970-01-01 00:00:00.227917815
4,1,0,64,3869200167,2760,25,1970-01-01 00:00:00.245916020


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [9]:
duration=data['Time_comb'].max()-data['Time_comb'].min()
duration

Timedelta('0 days 00:00:00.317463324')

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [11]:
data['max_nbr_channels']=data.groupby(['TDC_CHANNEL'])['TDC_MEAS'].transform(np.max)
data2=data.sort_values(by="max_nbr_channels", ascending=False)
print('Top 3 noisy channels',data2.loc[data2.index[0:3] , ['TDC_CHANNEL','max_nbr_channels']])

Top 3 noisy channels       TDC_CHANNEL  max_nbr_channels
0             123                29
7036           57                29
7029            4                29


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [40]:
df=data.groupby(['ORBIT_CNT']).count()['HEAD']
print('The number of non-empty orbits is:',len(df[df!=0]))

The number of non-empty orbits is: 111


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [13]:
print('The number of unique orbits with at least one measurement from TDC_CHANNEL=139 is')
len(data[ data['TDC_CHANNEL']==139 ]['ORBIT_CNT'].unique())

The number of unique orbits with at least one measurement from TDC_CHANNEL=139 is


111

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [16]:
series0=pd.Series(data.query('FPGA==0')['TDC_CHANNEL'],index=data.query('FPGA==0')['TDC_MEAS'])
series1=pd.Series(data.query('FPGA==1')['TDC_CHANNEL'],index=data.query('FPGA==1')['TDC_MEAS'])

In [19]:
series0

TDC_MEAS
26     59.0
27     59.0
28     61.0
19     53.0
25     62.0
      ...  
0     123.0
20      NaN
1     124.0
29     60.0
24     58.0
Name: TDC_CHANNEL, Length: 7714, dtype: float64

In [20]:
series1

TDC_MEAS
4      NaN
18     5.0
0      NaN
14     NaN
4      NaN
      ... 
6      NaN
11     NaN
20    10.0
0      NaN
14     NaN
Name: TDC_CHANNEL, Length: 3286, dtype: float64

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.